# Topic modeling analysis

#### Description
- All 11 qualitative questions analyzed for key topics
- Number of topics varied within the recommended range of 2 - 5 based on unique key words for each group. This was a subjective process that had to be completed by adjusting the number of components, examing the key words, looking at the intertopic distance in the visualizations and reading the most relevant responses to each topic.
- Non-cleaned text of must prominent topcs was included to make reading easier.

### Import libraries and data

In [1]:
import pandas as pd
import sys
import numpy as np
sys.path.append('../')


import nlp
import wrangle

import nltk

from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore")

In [2]:
pd.set_option('display.max_columns', None)
datadf, dictionarydf = wrangle.wrangle_data(path_prefix='../')

### Viewing the imported data

In [ ]:
datadf.head(1)

In [ ]:
datadf.describe()

In [ ]:
datadf.info()

# Analysis of qualitative questions

## 5. What is your company or organization's primary industry?

#### Clean and lemmatize the data for this question

In [ ]:
datadf.primary_industry = datadf.primary_industry.dropna().apply(nlp.basic_clean)
datadf.primary_industry = datadf.primary_industry.dropna().apply(nlp.lemmatize)

#### Create word count matrix and vector. Settings: 2 word ngrams permitted; words in more than 30% of documents were ignored.

In [ ]:
primary_industry_matrix, primary_industry_vector = nlp.create_wordcount_matrix(datadf.primary_industry.dropna(), ngram=(1,3), max_df=.3)
primary_industry_matrix, primary_industry_vector

#### Apply LDA method using 4, 6 and 8 components (can be changed) and a random state set to ensure the results can be replicated.

In [ ]:
lda5 = LatentDirichletAllocation(n_components= 8, random_state = 42)

In [ ]:
lda5.fit(primary_industry_matrix)

In [ ]:
pyLDAvis.sklearn.prepare(lda5, primary_industry_matrix, primary_industry_vector)

#### Topic groups (4)
1. education 
2. fintech 
3. healthcare 
4. Tech 

#### Topic groups (6) 
1. fintech
2. education
3. healthcare
4. technology
5. software
6. consultancy

#### Topic groups (8) - 5 & 8 slight overlap
1. fintech
2. education
3. technology
4. consulting
5. software
6. healthcare
7. government
8. public (sector)


In [ ]:
#8 Topics

In [ ]:
lda_W5 = lda5.transform(primary_industry_matrix)

In [ ]:
top_doc_column5 = datadf.primary_industry.dropna()

#### Create second word count matrix. Excludes words in appear in 80%+ of all documents and in 2 or fewer documents. Include bigrams.

In [ ]:
word_count_matrix5, count_vect5 = nlp.create_wordcount_matrix(datadf.primary_industry, max_df=0.8, min_df=2, ngram=(1,3))

In [ ]:
LDA5a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA5a.fit(word_count_matrix5)

In [ ]:
lda_H = LDA5a.transform(word_count_matrix5)

In [ ]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column5, 3)

## 6. What types of research do you currently use to make decisions?

In [ ]:
datadf.types_res_used = datadf.types_res_used.dropna().apply(nlp.basic_clean)
datadf.types_res_used = datadf.types_res_used.dropna().apply(nlp.lemmatize)

In [ ]:
types_res_used_matrix, types_res_used_vector = nlp.create_wordcount_matrix(datadf.types_res_used.dropna(), ngram=(1,3), max_df=.3)
types_res_used_matrix, types_res_used_vector

In [ ]:
lda6 = LatentDirichletAllocation(n_components= 8, random_state = 42)

In [ ]:
lda6.fit(types_res_used_matrix)

In [ ]:
pyLDAvis.sklearn.prepare(lda6, types_res_used_matrix, types_res_used_vector)

#### Topic groups (4)
1. card sort
2. contextual inquiry
3. focus group 
4. market rsearch

#### Topic groups (6) 
1. diary study
2. card (sort)
3. contextual inquiry
4. quantitative survey
5. focus (group)
6. generative (evaluative)

#### Topic groups (8) - 8 overlaps with 2 & 6
1. diary (study)
2. contextual inquiry
3. interview usability
4. gernative evaluative
5. market (research)
6. ux research
7. indepth (interview)
8. concept validation

In [ ]:
#8 Topics

In [ ]:
lda_W6 = lda6.transform(types_res_used_matrix)

In [ ]:
top_doc_column6 = datadf.types_res_used.dropna()

In [ ]:
word_count_matrix6, count_vect6 = nlp.create_wordcount_matrix(datadf.types_res_used.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [ ]:
LDA6a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA6a.fit(word_count_matrix6)


In [ ]:
lda_H = LDA6a.transform(word_count_matrix6)

In [ ]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column6, 3)

## 7. What types of research are you considering in the future?

In [ ]:
datadf.future_res = datadf.future_res.dropna().apply(nlp.basic_clean)
datadf.future_res = datadf.future_res.dropna().apply(nlp.lemmatize)

In [ ]:
future_res_matrix, future_res_vector = nlp.create_wordcount_matrix(datadf.future_res.dropna(), ngram=(1,3), max_df=.3)
future_res_matrix, future_res_vector

In [ ]:
lda7 = LatentDirichletAllocation(n_components=7, random_state = 42)

In [ ]:
lda7.fit(future_res_matrix)

In [ ]:
pyLDAvis.sklearn.prepare(lda7, future_res_matrix, future_res_vector)

#### Topic groups (4)
1. ab testing
2. quantitative
3. diary study
4. participatory

#### Topic groups (6) - 4 & 6 overlap
1. quantitative
2. usability testing
3. field
4. analytics
5. diary study
6. journey

#### Topic groups (8) - 5 is almost completely within 2; 6 & 8 overlap
1. quantitative
2. unmoderated usability
3. diary study
4. ab testing
5. ux
6. focus group
7. ethnographic
8. field study


In [ ]:
#8 topics

In [ ]:
lda_W7 = lda7.transform(future_res_matrix)

In [ ]:
top_doc_column7 = datadf.future_res.dropna()

In [ ]:
word_count_matrix7, count_vect7 = nlp.create_wordcount_matrix(datadf.future_res.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [ ]:
LDA7a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA7a.fit(word_count_matrix7)

In [ ]:
lda_H = LDA7a.transform(word_count_matrix7)

In [ ]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column7, 3)


## 10. Describe your educational background with research

In [ ]:
datadf.research_educ_desc = datadf.research_educ_desc.dropna().apply(nlp.basic_clean)
datadf.research_educ_desc = datadf.research_educ_desc.dropna().apply(nlp.lemmatize)

In [ ]:
research_educ_desc_matrix, research_educ_desc_vector = nlp.create_wordcount_matrix(datadf.research_educ_desc.dropna(), ngram=(1,3), max_df=.3)
research_educ_desc_matrix, research_educ_desc_vector

In [ ]:
lda10 = LatentDirichletAllocation(n_components= 4, random_state = 42)

In [ ]:
lda10.fit(research_educ_desc_matrix)

In [ ]:
pyLDAvis.sklearn.prepare(lda10, research_educ_desc_matrix, research_educ_desc_vector)

#### Topic groups (4) - no overlapping circles
1. participated
2. grad school
3. master degree
4. pyschology

#### Topic groups (6) - 1 &4, 2 & 3 overlap
1. participated
2. design research
3. social science
4. grad school
5. human factor
6. master degree

#### Topic groups (8) - extreme overlap between topics 1, 2, 4 & 8, 54% of all the responses

In [ ]:
#4 topics

In [ ]:
lda_W10 = lda10.transform(research_educ_desc_matrix)

In [ ]:
top_doc_column10 = datadf.research_educ_desc.dropna()

In [ ]:
word_count_matrix10, count_vect10 = nlp.create_wordcount_matrix(datadf.research_educ_desc.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [ ]:
LDA10a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA10a.fit(word_count_matrix10)

In [ ]:
lda_H = LDA10a.transform(word_count_matrix10)

In [ ]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column10, 3)

## 14. How do you decide which events to attend?

In [ ]:
datadf.how_pick_events = datadf.how_pick_events.dropna().apply(nlp.basic_clean)
datadf.how_pick_events = datadf.how_pick_events.dropna().apply(nlp.lemmatize)

In [ ]:
how_pick_events_matrix, how_pick_events_vector = nlp.create_wordcount_matrix(datadf.how_pick_events.dropna(), ngram=(1,3), max_df=.3)
how_pick_events_matrix, how_pick_events_vector

In [ ]:
lda14 = LatentDirichletAllocation(n_components= 6, random_state = 42)

In [ ]:
lda14.fit(how_pick_events_matrix)

In [ ]:
pyLDAvis.sklearn.prepare(lda14, how_pick_events_matrix, how_pick_events_vector)

#### Topic groups (4)
1. pay
2. topic
3. location
4. cost

#### Topic groups (6) - overlap between 3 & 6
1. pay
2. value
3. price
4. reputation
5. networking
6. relevance

#### Topic groups (8) - small amount of overlap among 2, 3, 4, & 6
1. design
2. affordable
3. speaker topic
4. reputation
5. location cost
6. value
7. location price
8. time away

In [ ]:
#6 topics

In [ ]:
lda_W14 = lda14.transform(how_pick_events_matrix)

In [ ]:
top_doc_column14 = datadf.how_pick_events.dropna()

In [ ]:
word_count_matrix14, count_vect14 = nlp.create_wordcount_matrix(datadf.how_pick_events.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [ ]:
LDA14a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA14a.fit(word_count_matrix14)

In [ ]:
lda_H = LDA14a.transform(word_count_matrix14)

In [ ]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column14, 3)

## 15. What was the best professional learning experience you've ever had?  What made it great?

In [66]:
best_event = datadf.best_event.fillna('nan').apply(nlp.basic_clean)
best_event = best_event.apply(nlp.lemmatize)

In [71]:
best_event_matrix, best_event_vector = nlp.create_wordcount_matrix(best_event, ngram=(1,3), max_df=.3, stop_words=stopWords)
best_event_matrix, best_event_vector

(<726x1630 sparse matrix of type '<class 'numpy.int64'>'
 	with 8977 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None,
                 stop_words=['nan', 'Nan', 'NaN', 'NAN', 'am', 'however',
                             'whatever', 'bottom', 'namely', 'most', 'whole',
                             'since', 'among', 'than', 'anyone', 'hers', 'every',
                             'yet', 'few', 'sixty', 'together', 'serious',
                             'latter', 'find', 'cant', 'until', 'none', 'up',
                             'he', 'meanwhile', ...],
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [72]:
lda15 = LatentDirichletAllocation(n_components= 4, random_state = 42)

In [73]:
lda15.fit(best_event_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=4, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [74]:
pyLDAvis.sklearn.prepare(lda15, best_event_matrix, best_event_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.000314  0.030749       1        1  30.552277
0     -0.007386  0.086046       2        1  29.044883
3     -0.090174 -0.064219       3        1  22.850552
1      0.097873 -0.052575       4        1  17.552287, topic_info=     Category       Freq         Term      Total  loglift  logprob
329   Default  58.000000          day  58.000000  30.0000  30.0000
1079  Default  10.000000      program  10.000000  29.0000  29.0000
1465  Default  22.000000      variety  22.000000  28.0000  28.0000
309   Default   9.000000      covered   9.000000  27.0000  27.0000
1231  Default  13.000000  small group  13.000000  26.0000  26.0000
...       ...        ...          ...        ...      ...      ...
1113   Topic4   6.639682         real  23.600379   0.4718  -5.5297
1378   Topic4   7.570581         time  39.089987   0.0984  -5.3985
1333   Topic4   8.709942         talk  77.441930  -0.4451  -5.2583
114    Topic4   7.384826     attended  48.795469  -0.1482  -5.4233
1438   Topic4   7.960541           ux  95.777908  -0.7475  -5.3482

[319 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
3         3  0.884711        12
10        3  0.201376      2013
10        4  0.604127      2013
31        1  0.404570      able
31        3  0.404570      able
...     ...       ...       ...
1617      2  0.451808      year
1617      3  0.107573      year
1617      4  0.086059      year
1618      1  0.346569  year ago
1618      2  0.635377  year ago

[471 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 4, 2])

#### Topic groups (4) - 3 is almost entirely enclosed by 2
1. research
2. networking
3. day
4. variety

#### Topic groups (6) - 2 intersects with 1 & 4
1. think
2. practical
3. learn
4. design
5. variety
6. intimate

#### Topic groups (8) - overlap between 1 & 2 and 3 & 4
1. strive
2. world
3. concept
4. uxpa
5. immediately
6. sxsw
7. relevant
8. start


In [ ]:
#To Be Decided

In [ ]:
lda_W15 = lda15.transform(best_event_matrix)

In [ ]:
top_doc_column15 = datadf.best_event.dropna()

In [ ]:
word_count_matrix15, count_vect15 = nlp.create_wordcount_matrix(datadf.best_event.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [ ]:
LDA15a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA15a.fit(word_count_matrix15)

In [ ]:
lda_H = LDA15a.transform(word_count_matrix15)

In [ ]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column15, 3)

## 16. What if any events have you attended on the subject of research in the past few years?

In [58]:
stop_words = ['nan', 'Nan', 'NaN', 'NAN']

stopWords = nlp.set_stop_words(stop_words)

In [59]:
events_attend_recent = datadf.events_attend_recent.fillna('nan')

In [60]:
events_attend_recent = events_attend_recent.astype('str')

In [61]:
events_attend_recent = events_attend_recent.apply(nlp.basic_clean)
events_attend_recent = events_attend_recent.apply(nlp.lemmatize)

In [62]:
events_attend_recent_matrix, events_attend_recent_vector = nlp.create_wordcount_matrix(events_attend_recent, ngram=(1,3), max_df=.3, stop_words=stopWords)

In [63]:
lda16 = LatentDirichletAllocation(n_components= 7, random_state = 42)

In [64]:
lda16.fit(events_attend_recent_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=7, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [65]:
pyLDAvis.sklearn.prepare(lda16, events_attend_recent_matrix, events_attend_recent_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.137903  0.012575       1        1  23.146554
3     -0.054751 -0.162626       2        1  18.343887
2     -0.061720 -0.044171       3        1  16.758350
6      0.089242  0.134281       4        1  12.092242
0     -0.043853  0.101827       5        1  11.393207
5     -0.005344  0.038405       6        1   9.465154
4      0.214330 -0.080291       7        1   8.800605, topic_info=    Category        Freq           Term       Total  loglift  logprob
102  Default  129.000000     conference  129.000000  30.0000  30.0000
516  Default   21.000000         strive   21.000000  29.0000  29.0000
598  Default   44.000000           uxpa   44.000000  28.0000  28.0000
282  Default   19.000000  local meetups   19.000000  27.0000  27.0000
407  Default   15.000000           qrca   15.000000  26.0000  26.0000
..       ...         ...            ...         ...      ...      ...
277   Topic7    5.052221          local   64.306009  -0.1135  -4.1332
9     Topic7    3.073809           2019   14.348182   0.8896  -4.6301
4     Topic7    2.993886           2018   15.987100   0.7551  -4.6565
84    Topic7    2.594062            chi   18.442834   0.4689  -4.7998
621   Topic7    2.001980       webinars   14.414305   0.4563  -5.0589

[399 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
2         1  0.198208   2017
2         2  0.198208   2017
2         3  0.099104   2017
2         5  0.297312   2017
2         7  0.198208   2017
...     ...       ...    ...
637       1  0.626201   year
637       2  0.144508   year
637       4  0.192677   year
637       5  0.048169   year
639       2  0.777038  young

[527 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 3, 7, 1, 6, 5])

# Topic groups (4)

1. day
2. london
3. epic
4. uxpa

#### Topic groups (6)
1. london
2. summit
3. session
4. design research
5. epic
6. uxr

#### Topic groups (8)
Heavy overlap among 6 of the 8 topics
1. local meetups
2. qrca
3. design research
4. uxpa
5. attended
6. epic
7. user research
8. focused


In [ ]:
#7 topics

In [ ]:
lda_W16 = lda16.transform(events_attend_recent_matrix)

In [ ]:
top_doc_column16 = datadf.events_attend_recent.dropna()

In [ ]:
word_count_matrix16, count_vect16 = nlp.create_wordcount_matrix(datadf.events_attend_recent.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [ ]:
LDA16a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA16a.fit(word_count_matrix16)

In [ ]:
lda_H = LDA16a.transform(word_count_matrix16)

In [ ]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column16, 3)

## 20. Did we miss any other types of conference sessions that you'd like to mention?

In [ ]:
other_conference_types = datadf.other_conference_types.fillna('nan').apply(nlp.basic_clean)
other_conference_types = other_conference_types.apply(nlp.lemmatize)

In [ ]:
other_conference_types.shape

In [ ]:
other_conference_types_matrix, other_conference_types_vector = nlp.create_wordcount_matrix(other_conference_types, ngram=(1,3), max_df=.3)
other_conference_types_matrix, other_conference_types_vector

In [ ]:
lda20 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [ ]:
lda20.fit(other_conference_types_matrix)

In [ ]:
pyLDAvis.sklearn.prepare(lda20, other_conference_types_matrix, other_conference_types_vector)

#### Topic groups (4)
1. case study
2. talk
3. poster session
4. nope

#### Topic groups (6)
1. reatreat
2. working
3. quality
4. case study
5. panel discussion
6. tutorial

#### Topic groups (8) - 3 is completely inside 1; 4 and 5 share about 75% of the same area
1. working
2. case study
3. nice
4. outside
5. variety
6. multitrack
7. panel
8. method

In [ ]:
#4 topics

In [ ]:
lda_W20 = lda20.transform(other_conference_types_matrix)

In [ ]:
top_doc_column20 = datadf.other_conference_types.dropna()

In [ ]:
word_count_matrix20, count_vect20 = nlp.create_wordcount_matrix(datadf.other_conference_types.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [ ]:
LDA20a = LatentDirichletAllocation(n_components=6, random_state=42)
LDA20a.fit(word_count_matrix20)

In [ ]:
lda_H = LDA20a.transform(word_count_matrix20)

In [ ]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column20, 3)

## 21. Subjects you most want to see covered at a research conference

In [3]:
stop_words = ['nan', 'Nan', 'NaN', 'NAN']

stopWords = nlp.set_stop_words(stop_words)

In [5]:
ideal_topics = datadf.ideal_topics.fillna('nan')
ideal_topics = ideal_topics.apply(nlp.basic_clean)
ideal_topics = ideal_topics.apply(nlp.lemmatize)

In [6]:
ideal_topics_matrix, ideal_topics_vector = nlp.create_wordcount_matrix(ideal_topics, ngram=(1,3), max_df=.3, stop_words=stopWords)
ideal_topics_matrix, ideal_topics_vector

(<726x1122 sparse matrix of type '<class 'numpy.int64'>'
 	with 5854 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None,
                 stop_words=['nan', 'Nan', 'NaN', 'NAN', 'am', 'however',
                             'whatever', 'bottom', 'namely', 'most', 'whole',
                             'since', 'among', 'than', 'anyone', 'hers', 'every',
                             'yet', 'few', 'sixty', 'together', 'serious',
                             'latter', 'find', 'cant', 'until', 'none', 'up',
                             'he', 'meanwhile', ...],
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [7]:
lda21 = LatentDirichletAllocation(n_components= 4, random_state = 42)

In [8]:
lda21.fit(ideal_topics_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=4, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [9]:
pyLDAvis.sklearn.prepare(lda21, ideal_topics_matrix, ideal_topics_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.148860 -0.008165       1        1  28.152875
1      0.076225  0.007506       2        1  26.018038
2      0.044807 -0.106360       3        1  23.396824
0      0.027827  0.107019       4        1  22.432263, topic_info=     Category        Freq          Term       Total  loglift  logprob
132   Default   51.000000          case   51.000000  30.0000  30.0000
989   Default   58.000000         study   58.000000  29.0000  29.0000
134   Default   43.000000    case study   43.000000  28.0000  28.0000
582   Default  127.000000        method  127.000000  27.0000  27.0000
663   Default   28.000000  organization   28.000000  26.0000  26.0000
...       ...         ...           ...         ...      ...      ...
349    Topic4    7.168547         field   23.810721   0.2942  -5.2699
244    Topic4    8.086925        design   43.977817  -0.1988  -5.1493
1041   Topic4    7.425933         topic   28.705437   0.1426  -5.2346
1012   Topic4    7.096921          team   48.230352  -0.4217  -5.2799
1016   Topic4    6.926207     technique   38.008794  -0.2078  -5.3043

[289 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
4         2  0.355751       academic
4         4  0.592918       academic
7         1  0.779334  accessibility
9         2  0.786195     accessible
12        1  0.197525     actionable
...     ...       ...            ...
1117      4  0.161048       workshop
1119      1  0.107872          world
1119      2  0.107872          world
1119      3  0.755101          world
1120      2  0.884274        writing

[453 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 3, 1])

#### Topic groups (4) - considerable overlap between 2 & 3
1. case study
2. finding
3. ops
4. new method

#### Topic groups (6)
1. case study
2. ops
3. stakeholder
4. new method
5. user research
6. quantitative

#### Topic groups (8)
1. case study
2. ops
3. user research
4. working
5. quant
6. mixed method
7. ethic
8. qualitative data


In [ ]:
#4 topics, 2.Finding/Data 3.ops/ai

In [ ]:
lda_W21 = lda21.transform(ideal_topics_matrix)

In [ ]:
top_doc_column21 = datadf.ideal_topics.dropna()

In [ ]:
word_count_matrix21, count_vect21 = nlp.create_wordcount_matrix(datadf.ideal_topics.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [ ]:
LDA21a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA21a.fit(word_count_matrix21)

In [ ]:
lda_H = LDA21a.transform(word_count_matrix21)

In [ ]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column21, 3)

## 22. If attending a conference about research, who might you be excited to see there?

In [49]:
stop_words = ['nan', 'Nan', 'NaN', 'NAN', 'research', 'conference', 'make', 'researcher', 'people', 'like', 'event', 'don']

words_to_stop = nlp.set_stop_words(stop_words)

In [50]:
words_to_stop

['nan',
 'Nan',
 'NaN',
 'NAN',
 'research',
 'conference',
 'make',
 'researcher',
 'people',
 'like',
 'event',
 'don',
 'am',
 'however',
 'whatever',
 'bottom',
 'namely',
 'most',
 'whole',
 'since',
 'among',
 'than',
 'anyone',
 'hers',
 'every',
 'yet',
 'few',
 'sixty',
 'together',
 'serious',
 'latter',
 'find',
 'cant',
 'until',
 'none',
 'up',
 'he',
 'meanwhile',
 'amoungst',
 'keep',
 'thin',
 're',
 'ourselves',
 'yourself',
 'via',
 'herself',
 'only',
 'either',
 'cannot',
 'anyhow',
 'herein',
 'both',
 'formerly',
 'anyway',
 'how',
 'never',
 'when',
 'though',
 'six',
 'him',
 'bill',
 'often',
 'whose',
 'below',
 'from',
 'full',
 'hereby',
 'nor',
 'everywhere',
 'even',
 'whoever',
 'name',
 'some',
 'everyone',
 'afterwards',
 'nowhere',
 'its',
 'was',
 'almost',
 'perhaps',
 'after',
 'at',
 'nothing',
 'then',
 'therefore',
 'former',
 'amount',
 'detail',
 'while',
 'although',
 'but',
 'any',
 'next',
 'sometime',
 'to',
 'is',
 'fifteen',
 'well',
 'ar

In [51]:
ideal_attendees = datadf.ideal_attendees.fillna('nan').apply(nlp.basic_clean)
ideal_attendees = ideal_attendees.apply(nlp.lemmatize)

In [52]:
ideal_att_matrix, ideal_att_vector = nlp.create_wordcount_matrix(ideal_attendees, ngram=(1,3), max_df=.3, stop_words=words_to_stop)
ideal_att_matrix, ideal_att_vector

(<726x796 sparse matrix of type '<class 'numpy.int64'>'
 	with 3740 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None,
                 stop_words=['nan', 'Nan', 'NaN', 'NAN', 'research',
                             'conference', 'make', 'researcher', 'people',
                             'like', 'event', 'don', 'am', 'however', 'whatever',
                             'bottom', 'namely', 'most', 'whole', 'since',
                             'among', 'than', 'anyone', 'hers', 'every', 'yet',
                             'few', 'sixty', 'together', 'serious', ...],
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [53]:
lda22 = LatentDirichletAllocation(n_components= 4, random_state = 42)

In [54]:
lda22.fit(ideal_att_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=4, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [55]:
pyLDAvis.sklearn.prepare(lda22, ideal_att_matrix, ideal_att_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.115550 -0.115501       1        1  29.433748
3     -0.077353  0.111970       2        1  25.302289
2      0.055586  0.063252       3        1  23.604723
1      0.137318 -0.059722       4        1  21.659240, topic_info=    Category       Freq        Term      Total  loglift  logprob
279  Default  15.000000        hall  15.000000  30.0000  30.0000
390  Default  54.000000      leader  54.000000  29.0000  29.0000
64   Default  27.000000         big  27.000000  28.0000  28.0000
104  Default  49.000000     company  49.000000  27.0000  27.0000
645  Default  15.000000     speaker  15.000000  26.0000  26.0000
..       ...        ...         ...        ...      ...      ...
767   Topic4   4.847684         way  13.251393   0.5241  -5.1633
667   Topic4   5.225445       steve  22.515172   0.0691  -5.0883
472   Topic4   4.881309         new  18.288811   0.2089  -5.1564
244   Topic4   4.303170        folk  16.582182   0.1808  -5.2825
319   Topic4   4.298879  indi young  12.432626   0.4678  -5.2835

[269 rows x 6 columns], token_table=      Topic      Freq                   Term
term                                        
4         1  0.659709               academic
4         2  0.164927               academic
4         3  0.164927               academic
7         1  0.896411  academic professional
15        4  0.913114                  admit
...     ...       ...                    ...
793       1  0.421781                  young
793       3  0.301272                  young
793       4  0.241018                  young
794       1  0.896409            young steve
795       1  0.896409   young steve portigal

[375 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 3, 2])

#### Topic groups (4)
1. industry
2. field
3. erika hall
4. Indi young

#### Topic groups (6) - some overlap between 2, 3, & 4; 5 is mostly overlapped with 2 & 4
1. steve portigal
2. expert
3. diverse
4. new
5. startup
6. doing research

#### Topic groups (8)
1. google
2. jared spool
3. different
4. new
5. consultant
6. erika hall
7. practioner
8. steve portigal


In [ ]:
#4 topics

In [ ]:
lda_W22 = lda22.transform(ideal_att_matrix)

In [ ]:
top_doc_column22 = datadf.ideal_attendees.dropna()

In [ ]:
word_count_matrix22, count_vect22 = nlp.create_wordcount_matrix(datadf.ideal_attendees.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [ ]:
LDA22a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA22a.fit(word_count_matrix22)

In [ ]:
lda_H = LDA22a.transform(word_count_matrix22)

In [ ]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column22, 3)

# Recommendations

In [41]:
stop_words = ['nan', 'Nan', 'NaN', 'NAN', 'dont', 'research', 'conference', 'make', 'researcher', 'people', 'like', 'event']

stopWords = nlp.set_stop_words(stop_words)

In [42]:
recommendations = datadf.recommendations.fillna('nan').apply(nlp.basic_clean)
recommendations = recommendations.apply(nlp.lemmatize)

In [43]:
recommendations_matrix, recommendations_vector = nlp.create_wordcount_matrix(recommendations, ngram=(1,3), max_df=.7, stop_words=stopWords)
recommendations_matrix, recommendations_vector

(<726x1211 sparse matrix of type '<class 'numpy.int64'>'
 	with 6262 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.7, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None,
                 stop_words=['nan', 'Nan', 'NaN', 'NAN', 'dont', 'research',
                             'conference', 'make', 'researcher', 'people',
                             'like', 'event', 'am', 'however', 'whatever',
                             'bottom', 'namely', 'most', 'whole', 'since',
                             'among', 'than', 'anyone', 'hers', 'every', 'yet',
                             'few', 'sixty', 'together', 'serious', ...],
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [44]:
lda23 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [45]:
lda23.fit(recommendations_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=3, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [46]:
pyLDAvis.sklearn.prepare(lda23, recommendations_matrix, recommendations_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.028141 -0.092411       1        1  40.121362
2     -0.071063  0.069115       2        1  31.624673
0      0.099204  0.023296       3        1  28.253965, topic_info=     Category       Freq     Term      Total  loglift  logprob
71    Default  35.000000   attend  35.000000  30.0000  30.0000
857   Default  16.000000     real  16.000000  29.0000  29.0000
1133  Default  28.000000       ux  28.000000  28.0000  28.0000
188   Default  22.000000  company  22.000000  27.0000  27.0000
341   Default  10.000000   europe  10.000000  26.0000  26.0000
...       ...        ...      ...        ...      ...      ...
203    Topic3   7.863693  content  23.734112   0.1593  -5.4898
385    Topic3   8.288480    focus  38.294930  -0.2665  -5.4372
627    Topic3   7.856657      lot  34.862259  -0.2261  -5.4907
588    Topic3   7.755276    level  25.377676   0.0784  -5.5037
548    Topic3   7.805293     just  58.841583  -0.7561  -5.4972

[235 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
5         3  0.959713  able attend
8         1  0.650734     academic
8         2  0.195220     academic
8         3  0.130147     academic
9         1  0.832053       access
...     ...       ...          ...
1200      3  0.510511        world
1206      1  0.875215      written
1207      1  0.310647         year
1207      2  0.051775         year
1207      3  0.673069         year

[331 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])

#### Topic groups (4)
1. pratical
2. affordable
3. expensive
4. levels

#### Topic groups (6) - some overlap between 2, 3, & 4; 5 is mostly overlapped with 2 & 4
1. ux
2. interesting
3. content
4. food
5. learning
6. affordalbe

#### Topic groups (8) - significant overlap amoung 6 of the 8 groups
1. events
2. ux
3. world
4. new
5. food
6. diverse
7. make accessible
8. north america

In [ ]:
#4 topics, 4. misc

In [ ]:
lda_W23 = lda23.transform(recommendations_matrix)

In [ ]:
top_doc_column23 = datadf.recommendations.dropna()

In [ ]:
word_count_matrix23, count_vect23 = nlp.create_wordcount_matrix(datadf.recommendations.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [ ]:
LDA23a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA23a.fit(word_count_matrix23)

In [ ]:
lda23.fit(recommendations_matrix)

In [ ]:
lda_H = LDA23a.transform(word_count_matrix23)

In [ ]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column23, 3)